# Linear elasticity: 2D cantilever under tip traction (plane strain)


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

 Geometry & mesh (rectangle with named sides)

In [2]:
shape = Rectangle(1.0, 1.0).Face()
shape.edges.Max(X).name = "right"
shape.edges.Min(X).name = "left"
shape.edges.Max(Y).name = "top"
shape.edges.Min(Y).name = "bottom"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.04))

Material (E, nu)

In [3]:
E, nu = 1.0, 0.20 
mu  = E/(2*(1+nu))
lam = E*nu/((1+nu)*(1-2*nu))    # 3D Lamé (plane strain default)

plane_stress = False
lam_eff = (2*mu*nu/(1-nu)) if plane_stress else lam


FE space (component-wise Dirichlet via a product space)

In [4]:
Vx = H1(mesh, order=2, dirichlet="left")     # only x-component fixed on 'left'
Vy = H1(mesh, order=2, dirichlet="bottom")   # only y-component fixed on 'bottom'
X  = Vx * Vy
(ux, uy), (vx, vy) = X.TnT()

Kinematics & constitutive law

In [5]:
def eps_2D(ux, uy):
    exx = grad(ux)[0]
    eyy = grad(uy)[1]
    exy = 0.5*(grad(ux)[1] + grad(uy)[0])
    return CoefficientFunction((exx, exy,   # row 0: [exx, exy]
                                exy, eyy),  # row 1: [exy, eyy]
                               dims=(2,2))

def sigma_2D(ux, uy):
    e  = eps_2D(ux, uy)
    tr = grad(ux)[0] + grad(uy)[1]           # exx + eyy
    return 2*mu*e + lam_eff*tr*Id(2)


Forms: a(u,v) and f(v)

In [ ]:
a = BilinearForm(X, symmetric=True)
a += InnerProduct(sigma_2D(ux, uy), eps_2D(vx, vy)) * dx

Uniform upward traction on the right edge (adjust as you like)

In [7]:
t = CoefficientFunction((1.0, 0.0))
f = LinearForm(X)
f += InnerProduct(t, CoefficientFunction((vx, vy))) * ds("right")

Solve

In [ ]:
with TaskManager():
    a.Assemble(); f.Assemble()
    gfu = GridFunction(X)
    gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Compute stress tensor and Von Mises stress for 2D (plane strain/stress)

In [9]:
uh = CoefficientFunction((gfu.components[0], gfu.components[1]))
s = sigma_2D(uh[0], uh[1])
von_mises = sqrt(s[0,0]**2 + s[1,1]**2 - s[0,0]*s[1,1] + 3*s[0,1]**2)

 Visualize displacement & Von Mises stress

In [10]:
uh = CoefficientFunction((gfu.components[0], gfu.components[1]))
norm_u = Norm(uh)
Draw(norm_u, mesh, "||u||", deformation=uh)
Draw(von_mises, mesh, "von Mises", deformation=uh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [11]:
vtk = VTKOutput(mesh, coefs=[norm_u, von_mises], names=["norm_u", "von_mises"], filename="elastic_lin").Do()

In [12]:
import pyvista as pv

pv.set_jupyter_backend('html')
visobj = pv.read('elastic_lin.vtu')

visobj.plot(scalars="norm_u", cpos="xy")
visobj.plot(scalars="von_mises", cpos="xy")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…